In [ ]:
# dynamically convert the data into that format.

In [3]:
from langchain.chains import create_extraction_chain_pydantic
from typing import List, Optional, Literal, Union
from pydantic import BaseModel, Field, HttpUrl
from enum import Enum
from neo4j import GraphDatabase
import uuid
import json
import re
import google.generativeai as genai
from google import genai
from google.genai import types
from google.genai.types import GenerateContentConfig, Part, SafetySetting
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Optional, List
from pydantic import BaseModel, Field, HttpUrl
from typing_extensions import Annotated, TypedDict

In [38]:
from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Any
from neo4j import GraphDatabase
import json
import uuid

In [9]:
from pydantic import BaseModel, Field
from typing import Optional, List

# Define all the provided Pydantic schemas first
class PriceWithCurrency(BaseModel):
    """Represents a price with its currency."""
    price: Optional[str] = Field(None, description="Example: '$10.00'")

class NetworkSpeeds(BaseModel):
    """Typical network speeds."""
    Typical_Download_Speed: Optional[str] = None
    Typical_Upload_Speed: Optional[str] = None
    Typical_Latency: Optional[str] = None

class OneTimeFee(BaseModel):
    """One-time fees associated with the plan."""
    activation_fee: Optional[PriceWithCurrency] = None
    federal_universal_service_fund: Optional[PriceWithCurrency] = None
    regulatory_cost_recovery: Optional[PriceWithCurrency] = None
    early_termination_fee: Optional[PriceWithCurrency] = None
    government_taxes: Optional[str] = None

class Pricing(BaseModel):
    """Pricing details of the plan."""
    base_price_usd: Optional[str] = None
    pricing_per_line_details_string: Optional[str] = None
    pricing_conditions: Optional[str] = None
    price_excludes_fees_taxes: Optional[str] = None
    fees_taxes_discounts_on_checkout: Optional[str] = None
    fees_taxes_surcharges_subject_to_change: Optional[str] = None
    one_time_fee: Optional[OneTimeFee] = None

class PriceLock(BaseModel):
    """Details about price lock guarantee."""
    price_lock_available: Optional[str] = None
    price_lock_duration_years: Optional[str] = None
    price_lock_scope: Optional[str] = None
    price_lock_exclusions: Optional[str] = None
    price_lock_void_conditions: Optional[str] = None

class Data(BaseModel):
    """Data allowances and policies."""
    data_unlimited: Optional[str] = None
    data_unlimited_clarification: Optional[str] = None
    data_premium_unlimited: Optional[str] = None
    data_allocation_total_gb: Optional[str] = None
    data_congestion_policy: Optional[str] = None
    data_congestion_policy_welcome_plan: Optional[str] = None
    data_carryover: Optional[str] = None
    data_carryover_condition: Optional[str] = None
    data_usage_clarification: Optional[str] = None

class TalkText(BaseModel):
    """Talk and text details."""
    talk_unlimited_domestic: Optional[str] = None
    text_unlimited_domestic: Optional[str] = None

class InternationalCalling(BaseModel):
    """Details about international calling features."""
    international_calling_included: Optional[str] = None
    international_calling_basic_included: Optional[str] = None
    international_calling_basic_destinations_count: Optional[str] = None
    international_calling_unlimited_destinations: Optional[str] = None
    international_calling_destinations_count: Optional[str] = None
    international_calling_destination_types: Optional[str] = None
    international_calling_destination_subject_to_change: Optional[str] = None
    international_calling_origin_restriction: Optional[str] = None
    international_calling_roaming_allowed: Optional[str] = None
    international_calling_usage_restriction: Optional[str] = None
    international_calling_billing_increment: Optional[str] = None
    international_calling_cost_structure: Optional[str] = None
    international_calling_rates_subject_to_change: Optional[str] = None
    international_calling_unique_numbers_limit: Optional[str] = None
    international_calling_unique_numbers_change_frequency: Optional[str] = None
    international_calling_other_conditions_apply: Optional[str] = None
    international_calling_global_choice_included: Optional[str] = None
    international_calling_global_choice_mexico_canada_unlimited: Optional[str] = None
    international_calling_global_choice_text_destinations_count: Optional[str] = None
    international_calling_global_choice_select_country_minutes_limit: Optional[str] = None
    international_calling_global_choice_select_country_count: Optional[str] = None

class InternationalTextRoaming(BaseModel):
    """Details about international text and roaming."""
    text_unlimited_international_destinations_count: Optional[str] = None
    international_roaming_mexico_canada_included: Optional[str] = None
    international_roaming_mexico_canada_data_limit_daily_gb: Optional[str] = None
    international_roaming_mexico_canada_throttle_speed: Optional[str] = None
    international_roaming_global_included: Optional[str] = None
    international_roaming_global_destinations_count: Optional[str] = None
    international_roaming_global_talk_unlimited: Optional[str] = None
    international_roaming_global_text_unlimited: Optional[str] = None
    international_roaming_global_data_limit_monthly_gb: Optional[str] = None
    international_roaming_global_throttle_speed_mbps: Optional[str] = None

class Hotspot(BaseModel):
    """Hotspot details."""
    hotspot_included: Optional[str] = None
    hotspot_unlimited_clarification: Optional[str] = None
    hotspot_limit_gb: Optional[str] = None
    hotspot_throttle_after_limit: Optional[str] = None
    hotspot_throttle_speed: Optional[str] = None
    hotspot_throttle_speed_mbps: Optional[str] = None
    hotspot_availability_condition: Optional[str] = None

class Network(BaseModel):
    """Network related information."""
    network_types_compatible: Optional[str] = None
    network_types_used: Optional[str] = None
    network_4glte_device_required: Optional[str] = None
    network_5g_requirements: Optional[str] = None
    network_5g_uwb_included: Optional[str] = None
    network_reliability_claim: Optional[str] = None
    network_promotional_claim: Optional[str] = None
    network_5g_uwb_speed_comparison: Optional[str] = None
    network_speed_claim_source: Optional[str] = None
    network_coverage_claim: Optional[str] = None
    network_variability_disclaimer: Optional[str] = None
    network_speeds_5G_Ultra_Wideband: Optional[NetworkSpeeds] = None
    network_speeds_5G: Optional[NetworkSpeeds] = None
    network_speeds_4G_LTE: Optional[NetworkSpeeds] = None

class VideoStreaming(BaseModel):
    """Video streaming quality details."""
    video_streaming_resolution_max: Optional[str] = None
    video_streaming_resolution_max_5guwb: Optional[str] = None

class Discounts(BaseModel):
    """Available discounts."""
    discount_acp_available: Optional[str] = None
    discount_acp_max_amount_usd: Optional[str] = None
    discount_acp_plan_eligibility: Optional[str] = None
    discount_acp_advance_purchase_limit: Optional[str] = None
    discount_lifeline_available: Optional[str] = None
    discount_lifeline_max_amount_usd: Optional[str] = None
    discount_lifeline_plan_eligibility: Optional[str] = None
    discount_lifeline_frequency_limit: Optional[str] = None
    discount_lifeline_advance_purchase_limit: Optional[str] = None
    discount_acp_lifeline_combined_available: Optional[str] = None
    discount_acp_lifeline_combined_max_amount_usd: Optional[str] = None
    discount_acp_lifeline_combined_plan_eligibility: Optional[str] = None
    discount_acp_lifeline_combined_frequency_limit: Optional[str] = None
    discount_acp_lifeline_combined_advance_purchase_limit: Optional[str] = None

class PromotionsAndPerks(BaseModel):
    """Promotions and additional perks."""
    promo_autorefill_available: Optional[str] = None
    promo_autorefill_discount_usd: Optional[str] = None
    promo_autorefill_discount_duration_months: Optional[str] = None
    promo_autorefill_eligibility: Optional[str] = None
    promo_autorefill_enrollment_requirements: Optional[str] = None
    promo_autorefill_plan_restriction: Optional[str] = None
    extended_plans_available: Optional[str] = None
    extended_plan_savings_details_string: Optional[str] = None
    promo_2x_data_mentioned: Optional[str] = None
    promo_free_phone_trade_in_available: Optional[str] = None
    promo_free_phone_trade_in_conditions: Optional[str] = None
    promo_kyop_credit_amount_usd: Optional[str] = None
    promo_kyop_credit_duration_months: Optional[str] = None
    promo_kyop_credit_termination_conditions: Optional[str] = None
    promo_kyop_stackable: Optional[str] = None
    promo_limited_time_offers_mentioned: Optional[str] = None
    device_promotions_available: Optional[str] = None
    device_promo_samsung_offer_details: Optional[str] = None
    device_promo_motorola_exclusion_details: Optional[str] = None

class FeaturesAndAddons(BaseModel):
    """Additional features and add-ons."""
    feature_enhanced_video_calling_streaming: Optional[str] = None
    feature_enhanced_video_calling_details: Optional[str] = None
    feature_enhanced_video_calling_availability_conditions: Optional[str] = None
    feature_verizon_family_included: Optional[str] = None
    feature_verizon_family_details: Optional[str] = None
    feature_call_filter_basic_included: Optional[str] = None
    feature_call_filter_plus_available: Optional[str] = None
    feature_ultimate_phone_upgrade_included: Optional[str] = None
    feature_ultimate_phone_upgrade_eligibility: Optional[str] = None
    feature_ultimate_phone_upgrade_benefit: Optional[str] = None
    feature_triple_minutes: Optional[str] = None
    perk_walmartplus_included: Optional[str] = None
    perk_walmartplus_condition: Optional[str] = None
    perk_walmartplus_eligibility_retention: Optional[str] = None
    perk_walmartplus_account_limit: Optional[str] = None
    perk_walmartplus_data_usage_applies: Optional[str] = None
    perk_walmartplus_offer_subject_to_change: Optional[str] = None
    perk_walmartplus_terms_apply: Optional[str] = None
    perk_walmartplus_benefits_list_string: Optional[str] = None

class OptionalPerks(BaseModel):
    """Optional perks that can be added to the plan."""
    optional_perks_available: Optional[str] = None
    optional_perks_price_range_usd_string: Optional[str] = None
    optional_perks_savings_claim: Optional[str] = None
    optional_perk_disney_hulu_espn_ads_available: Optional[str] = None
    optional_perk_netflix_max_ads_available: Optional[str] = None
    optional_perk_apple_one_available: Optional[str] = None
    optional_perk_apple_music_family_available: Optional[str] = None
    optional_perk_youtube_premium_available: Optional[str] = None
    optional_perk_100gb_hotspot_available: Optional[str] = None
    optional_perk_3_travelpass_days_available: Optional[str] = None
    optional_perk_unlimited_cloud_storage_available: Optional[str] = None
    optional_perk_google_one_ai_premium_available: Optional[str] = None
    optional_perk_second_number_available: Optional[str] = None
    optional_perks_plan_eligibility: Optional[str] = None
    optional_perks_billing_start: Optional[str] = None
    optional_perks_access_start_condition: Optional[str] = None
    optional_perks_age_residency_requirements: Optional[str] = None
    optional_perks_existing_subscriber_management_needed: Optional[str] = None
    optional_perks_streaming_upgrade_cost: Optional[str] = None
    optional_perks_apple_one_requirements: Optional[str] = None
    optional_perks_apple_one_subscription_management_needed: Optional[str] = None
    optional_perks_apple_one_apple_music_conflict: Optional[str] = None
    optional_perks_flexibility: Optional[str] = None
    optional_perks_cancellation_method: Optional[str] = None
    optional_perks_partner_terms_apply: Optional[str] = None

class KyopAndProtection(BaseModel):
    """Keep Your Own Phone and device protection details."""
    kyop_available: Optional[str] = None
    kyop_requirements: Optional[str] = None
    device_protection_available: Optional[str] = None
    device_protection_type: Optional[str] = None
    device_protection_requirements: Optional[str] = None
    device_protection_service_fee_max_usd: Optional[str] = None
    device_protection_claim_limit_period: Optional[str] = None
    device_protection_claim_limit_count: Optional[str] = None
    device_protection_min_msrp_usd: Optional[str] = None
    device_protection_preexisting_condition_exclusion: Optional[str] = None
    device_protection_other_exclusions_apply: Optional[str] = None
    device_protection_kyop_condition_required: Optional[str] = None
    device_protection_kyop_service_fee_usd: Optional[str] = None
    device_protection_kyop_replacement_variability: Optional[str] = None

class RewardsAndFamily(BaseModel):
    """Rewards program and family plan options."""
    rewards_program_available: Optional[str] = None
    rewards_use_points_for_plans: Optional[str] = None
    rewards_point_application_rule: Optional[str] = None
    rewards_points_cash_value: Optional[str] = None
    rewards_points_transferable: Optional[str] = None
    family_plan_available: Optional[str] = None
    family_plan_pricing_varied: Optional[str] = None
    family_plan_pricing_details_string: Optional[str] = None
    family_plan_discount_stackable_with_autopay: Optional[str] = None
    family_plan_device_promo_exclusion: Optional[str] = None

class AccountAndRefills(BaseModel):
    """Account management and refill options."""
    plan_mix_match_available: Optional[str] = None
    plan_mix_match_max_lines: Optional[str] = None
    plan_migration_rules: Optional[str] = None
    account_management_tools_available: Optional[str] = None
    account_management_features_string: Optional[str] = None
    refills_available: Optional[str] = None
    autorefill_available: Optional[str] = None
    autorefill_management_available: Optional[str] = None

class LegalAndPolicy(BaseModel):
    """Legal and policy information."""
    terms_and_conditions_reference: Optional[str] = None
    customer_type_eligibility: Optional[str] = None
    data_sharing_for_perks: Optional[str] = None

class Provider(BaseModel):
    """Information about the service provider."""
    provider_name: Optional[str] = None

class PlanDetails(BaseModel):
    """Details about the specific plan."""
    plan_name: Optional[str] = None
    plan_family: Optional[str] = None
    plan_type: Optional[str] = None
    plan_term_duration_days: Optional[str] = None
    contract_required: Optional[str] = None

class TelecomPlan(BaseModel):
    """
    A comprehensive Pydantic schema representing a telecom plan,
    unifying all related details from various categories.
    """
    provider: Optional[Provider] = None
    plan_details: Optional[PlanDetails] = None
    pricing: Optional[Pricing] = None
    price_lock: Optional[PriceLock] = None
    data: Optional[Data] = None
    talk_text: Optional[TalkText] = None
    international_calling: Optional[InternationalCalling] = None
    international_text_roaming: Optional[InternationalTextRoaming] = None
    hotspot: Optional[Hotspot] = None
    network: Optional[Network] = None
    video_streaming: Optional[VideoStreaming] = None
    discounts: Optional[Discounts] = None
    promotions_and_perks: Optional[PromotionsAndPerks] = None
    features_and_addons: Optional[FeaturesAndAddons] = None
    optional_perks: Optional[OptionalPerks] = None
    kyop_and_protection: Optional[KyopAndProtection] = None
    rewards_and_family: Optional[RewardsAndFamily] = None
    account_and_refills: Optional[AccountAndRefills] = None
    legal_and_policy: Optional[LegalAndPolicy] = None

In [11]:
import os
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "<API KEY>"

In [108]:
f = open("v2_straighttalk_platinum_unlimited.txt")
# f = open("straighttalk_platinum_unlimited.txt")
# f = open("tracphone_unlimited_talk.txt")

input_text_plan_details = f.read()
print(input_text_plan_details)


Based on the provided sources, here are the possible details related to Straight Talk's plan referred to as "Platinum Unlimited":

*   **Plan Name:** The plan is explicitly named **Platinum Unlimited**.
*   **Provider:** Straight Talk Wireless. Straight Talk is a registered trademark of Verizon Value, Inc., a Verizon company.

**Pricing**

*   The plan is associated with a potential price of **$65**. Specifically, the **Straight Talk Platinum $65 Unlimited plan** is mentioned in the context of the Walmart+ membership offer.
*   Discounts, taxes, and fees will be reflected on the checkout page.
*   Pricing varies depending on the number of lines in a Family Plan, although specific multi-line pricing for Platinum Unlimited is not provided in the sources.

**Network & Speed**

*   **Network (Specific Description):** For "All other customers" (distinguished from those potentially on the Verizon network as referenced elsewhere), the Platinum Unlimited plan uses an **LTE network**.
*   **Net

In [110]:
improved_system_message = """You are an assistant that converts broadband plan descriptions into structured JSON data.

- Use the Pydantic schema provided separately to map all fields correctly.
- Carefully extract relevant information from the input text.
- Populate all required fields according to the schema.
- Include optional fields only if the information is present.
- Use exact enum values where applicable.
- Maintain proper data types (numbers, strings, lists, booleans).
- Output strictly valid JSON conforming to the schema.
- Do not add explanations, comments, or any extra text—only output the JSON.
"""

In [118]:
# Initialize the model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=1)
structured_llm = llm.with_structured_output(TelecomPlan)

messages = [
    ("system", improved_system_message),
    ("human", input_text_plan_details),
]
# Invoke the model with a query asking for structured information
catalog_result = structured_llm.invoke(messages)

In [119]:
catalog_result

TelecomPlan(provider=Provider(provider_name='Straight Talk Wireless'), plan_details=PlanDetails(plan_name='Platinum Unlimited', plan_family=None, plan_type=None, plan_term_duration_days=None, contract_required=None), pricing=Pricing(base_price_usd='$65', pricing_per_line_details_string=None, pricing_conditions=None, price_excludes_fees_taxes=None, fees_taxes_discounts_on_checkout='Discounts, taxes, and fees will be reflected on the checkout page.', fees_taxes_surcharges_subject_to_change=None, one_time_fee=None), price_lock=None, data=Data(data_unlimited='true', data_unlimited_clarification=None, data_premium_unlimited=None, data_allocation_total_gb=None, data_congestion_policy=None, data_congestion_policy_welcome_plan=None, data_carryover=None, data_carryover_condition=None, data_usage_clarification=None), talk_text=TalkText(talk_unlimited_domestic='true', text_unlimited_domestic=None), international_calling=InternationalCalling(international_calling_included='true', international_cal

In [122]:
# Convert the result (which is a Pydantic model instance) to a JSON string
json_output = json.dumps(catalog_result.model_dump(), indent=2)

# Print or return the JSON
print(json_output)


{
  "provider": {
    "provider_name": "Straight Talk Wireless"
  },
  "plan_details": {
    "plan_name": "Platinum Unlimited",
    "plan_family": null,
    "plan_type": null,
    "plan_term_duration_days": null,
    "contract_required": null
  },
  "pricing": {
    "base_price_usd": "$65",
    "pricing_per_line_details_string": null,
    "pricing_conditions": null,
    "price_excludes_fees_taxes": null,
    "fees_taxes_discounts_on_checkout": "Discounts, taxes, and fees will be reflected on the checkout page.",
    "fees_taxes_surcharges_subject_to_change": null,
    "one_time_fee": null
  },
  "price_lock": null,
  "data": {
    "data_unlimited": "true",
    "data_unlimited_clarification": null,
    "data_premium_unlimited": null,
    "data_allocation_total_gb": null,
    "data_congestion_policy": null,
    "data_congestion_policy_welcome_plan": null,
    "data_carryover": null,
    "data_carryover_condition": null,
    "data_usage_clarification": null
  },
  "talk_text": {
    "talk

In [124]:
file_path = "v2_straighttalk_platinum_unlimited.json"
with open(file_path, "w") as file:
    json.dump(json_output, file, indent=2)

In [68]:
class Neo4jConverter:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        self.driver.close()

    def _clean_properties(self, properties: Dict[str, Any]) -> Dict[str, Any]:
        """
        Cleans properties dictionary by removing None values and handling PriceWithCurrency.
        """
        cleaned_properties = {}
        for key, value in properties.items():
            if value is not None:
                if isinstance(value, PriceWithCurrency):
                    cleaned_properties[key] = value.__root__
                else:
                    cleaned_properties[key] = value
        return cleaned_properties

    def _create_node_and_relationship(self, tx, node_label: str, node_props: Dict[str, Any],
                                      parent_node_label: str, parent_node_props: Dict[str, Any],
                                      relationship_type: str):
        """
        Helper to create or merge a node based on its properties and link it to a parent node.
        """
        cleaned_node_props = self._clean_properties(node_props)
        cleaned_parent_props = self._clean_properties(parent_node_props)

        if not cleaned_node_props:
            print(f"Warning: Attempted to create {node_label} node with no meaningful properties. Skipping.")
            return None

        # Construct parameter names for node properties and the MERGE clause
        node_prop_params = {f"node_{k}": v for k, v in cleaned_node_props.items()}
        node_match_clause = ", ".join([f"{k}: $node_{k}" for k in cleaned_node_props.keys()])

        # Construct parameter names for parent properties and the MATCH clause
        parent_prop_params = {f"parent_{k}": v for k, v in cleaned_parent_props.items()}
        parent_match_clause = ", ".join([f"{k}: $parent_{k}" for k in cleaned_parent_props.keys()])

        # Combine all parameters for the transaction
        all_params = {**node_prop_params, **parent_prop_params}

        query = (
            f"MERGE (n:{node_label} {{{node_match_clause}}})"
            f" WITH n"
            f" MATCH (p:{parent_node_label} {{{parent_match_clause}}})"
            f" MERGE (p)-[:{relationship_type}]->(n)"
            f" RETURN n"
        )
        result = tx.run(query, **all_params)
        return result.single()[0] # Return the created/merged node

    def convert_telecom_plan_to_neo4j(self, plan_data: Dict[str, Any]):
        """
        Converts a dictionary representing a TelecomPlan into Neo4j nodes and relationships,
        merging nodes based on their properties.
        """
        try:
            telecom_plan = TelecomPlan(**plan_data)
        except Exception as e:
            print(f"Error validating TelecomPlan data with Pydantic: {e}")
            return

        with self.driver.session() as session:
            # 1. Prepare properties for the main TelecomPlan node
            main_plan_props = {}
            if telecom_plan.provider:
                main_plan_props.update(telecom_plan.provider.dict(exclude_none=True))
            if telecom_plan.plan_details:
                main_plan_props.update(telecom_plan.plan_details.dict(exclude_none=True))

            cleaned_main_plan_props = self._clean_properties(main_plan_props)

            if not cleaned_main_plan_props:
                print("Error: Main TelecomPlan node has no properties to merge on. Skipping.")
                return

            # Construct parameter names for main plan properties and the MERGE clause
            main_plan_prop_params = {f"main_{k}": v for k, v in cleaned_main_plan_props.items()}
            main_plan_match_clause = ", ".join([f"{k}: $main_{k}" for k in cleaned_main_plan_props.keys()])

            # Execute the MERGE for the main TelecomPlan node
            result = session.write_transaction(
                lambda tx, props_params, match_clause: tx.run(
                    f"MERGE (tp:TelecomPlan {{{match_clause}}}) RETURN tp",
                    **props_params
                ).single()[0],
                main_plan_prop_params, main_plan_match_clause
            )
            telecom_plan_node = result # This is the actual Neo4j Node object

            # The properties used to MERGE the TelecomPlan node serve as its unique identifier
            # for linking other nodes.
            parent_node_props_for_linking = cleaned_main_plan_props

            # 2. Iterate through other top-level optional sub-schemas and create nodes/relationships
            # Exclude 'provider' and 'plan_details' as their properties are now on the main node
            sub_schema_map = {
                "pricing": ("Pricing", "HAS_PRICING"),
                "price_lock": ("PriceLock", "HAS_PRICE_LOCK"),
                "data": ("Data", "HAS_DATA"),
                "talk_text": ("TalkText", "HAS_TALK_TEXT"),
                "international_calling": ("InternationalCalling", "HAS_INTERNATIONAL_CALLING"),
                "international_text_roaming": ("InternationalTextRoaming", "HAS_INTERNATIONAL_ROAMING"),
                "hotspot": ("Hotspot", "HAS_HOTSPOT"),
                "network": ("Network", "HAS_NETWORK"),
                "video_streaming": ("VideoStreaming", "HAS_VIDEO_STREAMING"),
                "discounts": ("Discounts", "HAS_DISCOUNTS"),
                "promotions_and_perks": ("PromotionsAndPerks", "HAS_PROMOTIONS_AND_PERKS"),
                "features_and_addons": ("FeaturesAndAddons", "HAS_FEATURES_AND_ADDONS"),
                "optional_perks": ("OptionalPerks", "HAS_OPTIONAL_PERKS"),
                "kyop_and_protection": ("KyopAndProtection", "HAS_KYOP_AND_PROTECTION"),
                "rewards_and_family": ("RewardsAndFamily", "HAS_REWARDS_AND_FAMILY"),
                "account_and_refills": ("AccountAndRefills", "HAS_ACCOUNT_AND_REFILLS"),
                "legal_and_policy": ("LegalAndPolicy", "HAS_LEGAL_AND_POLICY"),
            }

            for attr_name, (label, rel_type) in sub_schema_map.items():
                sub_data = getattr(telecom_plan, attr_name)
                if sub_data:
                    sub_properties = sub_data.dict(exclude_none=True)

                    # Handle nested structures like OneTimeFee within Pricing
                    if attr_name == "pricing" and sub_data.one_time_fee:
                        one_time_fee_properties = sub_data.one_time_fee.dict(exclude_none=True)
                        sub_properties.pop("one_time_fee", None) # Remove from parent's properties

                        # Create Pricing node first
                        pricing_node = session.write_transaction(
                            self._create_node_and_relationship, label, sub_properties, "TelecomPlan", parent_node_props_for_linking, rel_type
                        )
                        if pricing_node: # Only proceed if pricing_node was successfully created/merged
                            # Create OneTimeFee node and link to Pricing
                            session.write_transaction(
                                self._create_node_and_relationship, "OneTimeFee", one_time_fee_properties, label, sub_properties, "HAS_ONE_TIME_FEE"
                            )
                    # Handle nested structures like NetworkSpeeds within Network
                    elif attr_name == "network" and (sub_data.network_speeds_5G_Ultra_Wideband or sub_data.network_speeds_5G or sub_data.network_speeds_4G_LTE):
                        network_speeds_data = {
                            "network_speeds_5G_Ultra_Wideband": sub_data.network_speeds_5G_Ultra_Wideband,
                            "network_speeds_5G": sub_data.network_speeds_5G,
                            "network_speeds_4G_LTE": sub_data.network_speeds_4G_LTE
                        }
                        # Remove network_speeds from network_properties to create them as separate nodes
                        sub_properties.pop("network_speeds_5G_Ultra_Wideband", None)
                        sub_properties.pop("network_speeds_5G", None)
                        sub_properties.pop("network_speeds_4G_LTE", None)

                        # Create Network node first
                        network_node = session.write_transaction(
                            self._create_node_and_relationship, label, sub_properties, "TelecomPlan", parent_node_props_for_linking, rel_type
                        )

                        if network_node: # Only proceed if network_node was successfully created/merged
                            # Create NetworkSpeeds nodes and link to Network
                            for speed_type, speed_data in network_speeds_data.items():
                                if speed_data:
                                    speed_label = speed_type.replace("network_speeds_", "") # e.g., "5G_Ultra_Wideband"
                                    session.write_transaction(
                                        self._create_node_and_relationship, f"NetworkSpeeds_{speed_label}", speed_data.dict(exclude_none=True), label, sub_properties, f"HAS_SPEEDS_{speed_label.upper()}"
                                    )
                    else:
                        # For all other top-level sub-schemas
                        session.write_transaction(
                            self._create_node_and_relationship, label, sub_properties, "TelecomPlan", parent_node_props_for_linking, rel_type
                        )
        print(f"Telecom Plan '{cleaned_main_plan_props.get('plan_name', 'Unnamed')}' by '{cleaned_main_plan_props.get('provider_name', 'Unknown')}' successfully converted to Neo4j.")


In [126]:
with open("v2_straighttalk_platinum_unlimited.json") as f:
    upload_json_data = json.load(f)
print(upload_json_data)

{
  "provider": {
    "provider_name": "Straight Talk Wireless"
  },
  "plan_details": {
    "plan_name": "Platinum Unlimited",
    "plan_family": null,
    "plan_type": null,
    "plan_term_duration_days": null,
    "contract_required": null
  },
  "pricing": {
    "base_price_usd": "$65",
    "pricing_per_line_details_string": null,
    "pricing_conditions": null,
    "price_excludes_fees_taxes": null,
    "fees_taxes_discounts_on_checkout": "Discounts, taxes, and fees will be reflected on the checkout page.",
    "fees_taxes_surcharges_subject_to_change": null,
    "one_time_fee": null
  },
  "price_lock": null,
  "data": {
    "data_unlimited": "true",
    "data_unlimited_clarification": null,
    "data_premium_unlimited": null,
    "data_allocation_total_gb": null,
    "data_congestion_policy": null,
    "data_congestion_policy_welcome_plan": null,
    "data_carryover": null,
    "data_carryover_condition": null,
    "data_usage_clarification": null
  },
  "talk_text": {
    "talk

In [58]:
NEO4J_URI = "neo4j+s://dadc39f9.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "<API KEY>"

In [128]:
converter = Neo4jConverter(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)
converter.convert_telecom_plan_to_neo4j(json.loads(upload_json_data))
converter.close()

C:\Users\krish\AppData\Local\Temp\ipykernel_17004\3821692553.py:70: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  main_plan_props.update(telecom_plan.provider.dict(exclude_none=True))
C:\Users\krish\AppData\Local\Temp\ipykernel_17004\3821692553.py:72: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  main_plan_props.update(telecom_plan.plan_details.dict(exclude_none=True))
C:\Users\krish\AppData\Local\Temp\ipykernel_17004\3821692553.py:85: DeprecationWarning: write_transaction has been renamed to execute_write
  result = session.write_transaction(
C:\Users\krish\AppData\Local\Temp\ipykernel_17004\3821692553.py:123: PydanticDeprecatedSince20: The `dict`

Telecom Plan 'Platinum Unlimited' by 'Straight Talk Wireless' successfully converted to Neo4j.


In [52]:
upload_json_data

'{\n  "provider": {\n    "provider_name": "Verizon"\n  },\n  "plan_details": {\n    "plan_name": "Unlimited Ultimate",\n    "plan_family": "myPlan",\n    "plan_type": "Prepaid",\n    "plan_term_duration_days": null,\n    "contract_required": null\n  },\n  "pricing": {\n    "base_price_usd": "65",\n    "pricing_per_line_details_string": "For 4 lines: $65 / line. For 3 lines: $75 / line. For 2 lines: $90 / line. For 1 line: $100 / line.",\n    "pricing_conditions": null,\n    "price_excludes_fees_taxes": "yes",\n    "fees_taxes_discounts_on_checkout": "A $10/mo discount is available when using Auto Pay with ACH or the Verizon Visa Card and paper-free billing.",\n    "fees_taxes_surcharges_subject_to_change": null,\n    "one_time_fee": null\n  },\n  "price_lock": {\n    "price_lock_available": "yes",\n    "price_lock_duration_years": "3",\n    "price_lock_scope": "base monthly rate for talk, text, and data",\n    "price_lock_exclusions": "taxes, fees, surcharges, additional plan discounts